In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("../src")
sys.path.append("../src/qspright")

import rna_transform.utils as utils
from rna_transform.rna_utils import RNAHelper
import itertools

# import tqdm and solve a visualisation problem with tqdm
from tqdm import tqdm
from functools import partial
tqdm = partial(tqdm, position=0, leave=True)

# set parallelism
parallel = True

if parallel:
    from multiprocessing import Pool

ImportError: cannot import name 'RNAHelper'

In [ ]:
positions = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

helper = RNAHelper(positions)
n = helper.n
q = helper.q

print("n = {}, N = {}".format(n, q ** n))

## Calculate free energies of different RNA sequences

Constructs and save the data corresponding to the quasi-empirical RNA fitness function of the Hammerhead ribozyme HH9.

In [ ]:
# helper.calculate_rna_data(parallel=parallel)

### Calculate true values of the GWHT coefficients

Calculates the GWHT transform of the RNA free energy values using vanilla FFT.

In [ ]:
emp_beta_gwht = helper.calculate_rna_gwht(save=False)

In [ ]:
# Calculate fraction of variance explained by empirical coefficients
bm_fv_gwht = utils.calc_frac_var_explained(emp_beta_gwht)

In [ ]:
plt.plot(bm_fv_gwht[:1000], label = "GWHT")
plt.legend()
plt.show()

## Test QSPRIGHT with different parameters

Construct a grid of parameters. For each entry, run multiple test rounds. Compute the average for each parameter selection. 

In [ ]:
iters = 11
num_subsample_list = [2, 3, 4, 5]
num_random_delays_list = [3, 5, 7, 9]
b_list = [4, 5, 6, 7]
test_params_list = list(itertools.product(num_subsample_list, num_random_delays_list, b_list))

def _gwht_test(i):
    param_idx = i % len(test_params_list)
    test_params = test_params_list[param_idx]
    num_subsample, num_random_delays, b = test_params
    gwht, (n_used, n_used_unique), peeled = helper.calculate_rna_qspright(save=False, noise_sd = 800 / (q ** n), report = True, num_subsample = num_subsample, num_random_delays = num_random_delays, b = b)
    emp_beta_qspright = np.reshape(gwht, -1)
    sample_ratio = n_used / q ** n
    unique_sample_ratio = n_used_unique/ q ** n
    nmse = np.sum(np.abs(emp_beta_gwht - emp_beta_qspright)**2) / np.sum(np.abs(emp_beta_gwht)**2)
    #print(len(peeled), nmse)
    return param_idx, sample_ratio, unique_sample_ratio, nmse

In [ ]:
if parallel:
    
    exp_count = iters * len(test_params_list)

    with Pool() as pool:
        pred = list(tqdm(pool.imap(_gwht_test, range(exp_count)), total=exp_count))

else:
    # TODO
    raise NotImplementedError("non-parallel version is not implemented")

In [ ]:
test_params_idx_list = list(itertools.product(np.arange(len(num_subsample_list)), np.arange(len(num_random_delays_list)), np.arange(len(b_list))))

sample_ratios = np.zeros((len(num_subsample_list), len(num_random_delays_list), len(b_list), iters))
unique_sample_ratios = np.zeros((len(num_subsample_list), len(num_random_delays_list), len(b_list), iters))
nmses = np.zeros((len(num_subsample_list), len(num_random_delays_list), len(b_list), iters))

for i, exp_result in enumerate(pred):
    iter_idx = i // len(test_params_list)
    test_params_idx = i % len(test_params_list)
    test_params_idx = test_params_idx_list[test_params_idx]
    param_idx, sample_ratio, unique_sample_ratio, nmse = exp_result
    idx = test_params_idx + (iter_idx,)
    sample_ratios[idx] = sample_ratio
    unique_sample_ratios[idx] = unique_sample_ratio
    nmses[idx] = nmse

In [ ]:
for i, b in enumerate(b_list):
    s_values = np.mean(unique_sample_ratios[:, :, i, :], axis=-1).flatten()
    mse_values =  np.mean(nmses[:, :, i, :], axis=-1).flatten()

    plt.scatter(s_values, mse_values, label = "b = {}".format(b))

plt.xlabel('Unique Sample Ratio')
plt.ylabel('NMSE')
plt.legend()
plt.show()

In [ ]:
for i, b in enumerate(b_list):
    s_values = np.mean(sample_ratios[:, :, i, :], axis=-1).flatten()
    mse_values =  np.mean(nmses[:, :, i, :], axis=-1).flatten()

    plt.scatter(s_values, mse_values, label = "b = {}".format(b))

plt.xlabel('Total Sample Ratio')
plt.ylabel('NMSE')
plt.legend()
plt.show()